In [ ]:
import requests
from kafka import KafkaProducer
import json
import time
import re

# Configuration du Kafka Producer
kafka_broker = "kafka:9092"
kafka_topic = "pole-emploi"

producer = KafkaProducer(bootstrap_servers=kafka_broker,
                         value_serializer=lambda m: json.dumps(m).encode('ascii'))

# Les informations d'authentification pour l'API
client_id = "PAR_ynov_3141bf157b6bfff2eaae5b6e2b056f0502707b9b7056bcbe00acdf8224fff2b1"
client_secret = "9dbcf2bb44219efa2cd94e1dbf397f3f3cb3266e6cb3b2174d720f127bfc851b"

# Les informations de recherche pour les offres d'emploi
offre_url = "https://api.emploi-store.fr/partenaire/offresdemploi/v2/offres/search"
offre_params = {
    "motsCles": "Informatique",
    "lieu": "France",
    "sort": "0"
}

# Obtenir le jeton d'accès en utilisant OAuth2
token_url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
token_payload = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "application_"+client_id+" api_offresdemploiv2 o2dsoffre"
}
token_response = requests.post(token_url, data=token_payload)
access_token = token_response.json()["access_token"]

# Envoyer la requête de recherche pour les offres d'emploi
offre_headers = {
    "Authorization": f"Bearer {access_token}"
}

while True:
    response = requests.get(offre_url, params=offre_params, headers=offre_headers)
    if response.status_code == 206:
        data = response.content.decode() # Décode les données en tant que chaîne de caractères
        producer.send(kafka_topic, data)
        range_header = response.headers.get('Content-Range')
        if range_header:
            match = re.match(r'^bytes (\d+)-(\d+)/(\d+)', range_header)
            if match:
                start = int(match.group(2)) + 1
                end = min(int(match.group(3)), start + 99999)
                offre_headers['Range'] = f'bytes={start}-{end}'
        time.sleep(5)
    elif response.status_code == 200:
        data = response.json()
        producer.send(kafka_topic, data)
        time.sleep(5)
    else:
        print("Erreur lors de la requête : {}".format(response.status_code))
        break
